In [1]:
import numpy as np
import itertools

In [2]:
c = 3

In [3]:
# Generate some data
np.random.seed(42)
lambda1 = np.random.normal(size=(c, c))
lambda2 = np.random.normal(size=(c, c))
lambda3 = np.random.normal(size=(c, c))
G1 = np.random.normal(size=(c, c, c))
G2 = np.random.normal(size=(c, c, c))
U = np.random.normal(size=(c, c, c, c))

In [4]:
def Z_naive(lambda1, lambda2, lambda3, G1, G2, U):
    c = lambda1.shape[0]
    Z = np.zeros(shape=(c, c, c, c))
    for a, b, c, d, e, f, g, h, i, j in itertools.product(*([range(c)]*10)):
        Z[a, h, i, j] += lambda1[a, b]*lambda2[d, e]*lambda3[g, h]*G1[c, b, d]*G2[f, e, g]*U[i, j, c, f]
    return Z

In [5]:
%%timeit
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)

47.8 ms ± 218 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
Z = Z_naive(lambda1, lambda2, lambda3, G1, G2, U)
Z.shape

(3, 3, 3, 3)

In [14]:
%%timeit
Z=np.einsum('ab, cbd, de, feg, gh, ijcf -> ahij',
            lambda1, G1, lambda2, G2, lambda3, U,
            optimize = True)
    

137 µs ± 989 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [15]:
information=np.einsum_path('ab, cbd, de, feg, gh, ijcf -> ahij',
                         lambda1, G1, lambda2, G2, lambda3, U,
                         optimize = True)
print(information[1])

  Complete contraction:  ab,cbd,de,feg,gh,ijcf->ahij
         Naive scaling:  10
     Optimized scaling:  6
      Naive FLOP count:  3.543e+05
  Optimized FLOP count:  2.431e+03
   Theoretical speedup:  145.740
  Largest intermediate:  8.100e+01 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   4                 cbd,ab->acd                 de,feg,gh,ijcf,acd->ahij
   4                 feg,de->dfg                    gh,ijcf,acd,dfg->ahij
   4                 dfg,gh->dfh                       ijcf,acd,dfh->ahij
   5               dfh,acd->acfh                          ijcf,acfh->ahij
   6             acfh,ijcf->ahij                               ahij->ahij
